# ACRF: GLM Analysis for VASCO Transients

**@sara0021 | Oct 29, 2025**

Fits Binomial GLM, plots 2-3x spikes.

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

# Load
df = pd.read_csv('../data/transients.csv')
df['date'] = pd.to_datetime(df['date'])
print(f"Loaded {len(df)} days. Baseline: {df['transient_count'].mean():.1f}")
df.head()

In [ ]:
# X/y
X = df[['nuclear_event', 'uap_event', 'geomag_kp']]
X = sm.add_constant(X)
y = df['transient_count']
print("Ready.")

In [ ]:
# GLM
glm = sm.GLM(y, X, family=sm.families.Binomial())
result = glm.fit()
print(result.summary())

coeffs = result.params
pvals = result.pvalues
print(f"\nNuclear: exp({coeffs['nuclear_event']:.3f}) ≈ {np.exp(coeffs['nuclear_event']):.1f}x (p={pvals['nuclear_event']:.3e})")
print(f"UAP: exp({coeffs['uap_event']:.3f}) ≈ {np.exp(coeffs['uap_event']):.1f}x (p={pvals['uap_event']:.3e})")

In [ ]:
# Plots
plt.figure(figsize=(12,5))
plt.bar(df['date'], df['transient_count'], color='lightblue', label='Daily')
event_mask = (df['nuclear_event']==1) | (df['uap_event']==1)
plt.bar(df[event_mask]['date'], df[event_mask]['transient_count'], color='red', label='Event (2-3x)')
plt.axhline(df['transient_count'].mean(), color='green', linestyle='--', label='Baseline')
plt.title('VASCO: 2-3x Spikes')
plt.xlabel('Date'); plt.ylabel('Count'); plt.legend(); plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../results/spikes.png', dpi=150)
plt.show()

df['event_day'] = event_mask.astype(int)
df.boxplot(column='transient_count', by='event_day', figsize=(6,4))
plt.title('Event vs Non (2-3x)')
plt.suptitle(''); plt.ylabel('Count')
plt.show()
print(f"Event: {df[df['event_day']==1]['transient_count'].mean():.1f} ({df[df['event_day']==1]['transient_count'].mean()/df[df['event_day']==0]['transient_count'].mean():.1f}x)")